The following exercises are meant to be solved by gathering the bash commands incrimentally in two scripts, one for ex 1.* the other for ex 2.* 

### Ex 1

1\.a Make a new directory called `students` in your home. Download a csv file with the list of students of this lab from [here](https://www.dropbox.com/s/867rtx3az6e9gm8/LCP_22-23_students.csv) (use the `wget` command) and copy that to `students`. First check whether the file is already there

1\.b Make two new files, one containing the students belonging to PoD, the other to Physics.

1\.c For each letter of the alphabet, count the number of students whose surname starts with that letter. 

1\.d Find out which is the letter with most counts.

1\.e Assume an obvious numbering of the students in the file (first line is 1, second line is 2, etc.), group students "modulo 18", i.e. 1,19,37,.. 2,20,38,.. etc. and put each group in a separate file  

In [ ]:
#!/bin/bash

# 1.a Create a directory called "students" in your home directory
mkdir ~/students

# Check if the CSV file is already present in the "students" directory
if [ -e ~/students/students.csv ]; then
    echo "CSV file already exists."
else
    # Download the CSV file from the provided URL using wget
    wget -O ~/students/students.csv https://www.dropbox.com/s/867rtx3az6e9gm8/LCP_22-23_students.csv
fi

# 1.b Extract students belonging to PoD and Physics into separate files
grep "PoD" ~/students/students.csv > ~/students/pod_students.csv
grep "Physics" ~/students/students.csv > ~/students/physics_students.csv

# 1.c Count students by the first letter of their surname
for letter in {A..Z}; do
    count=$(grep -c "^$letter" ~/students/students.csv)
    echo "$letter: $count"
done

# 1.d Find the letter with the most counts
max_count=0
max_letter=""
for letter in {A..Z}; do
    count=$(grep -c "^$letter" ~/students/students.csv)
    if [ $count -gt $max_count ]; then
        max_count=$count
        max_letter=$letter
    fi
done
echo "The letter with the most counts is $max_letter with $max_count students."

file_length=$(wc -l < ~/students/students.csv)
echo "file length: $file_length"
for ((i=0; i<$file_length; i++))
do
echo "i:$i"
spec=$(($i % 18))
echo "spec:$spec"
outfile="output${spec}.csv"
echo "$outfile"
line=$(($i + 1))
echo "line:$line" 
sed -n "${line}p" ~/students/students.csv >> "$outfile"
done
# 1.e Group students by "modulo 18" and save each group in separate files
#for ((i = 1; i <= 18; i++)); do
#    group_file="~/students/group_$i.csv"
#    awk -v group="$i" 'NR % 18 == group' ~/students/students.csv > "$group_file"
#done


### Ex 2

2.a Make a copy of the file `data.csv` removing the metadata and the commas between numbers; call it `data.txt`

2\.b How many even numbers are there?

2\.c Distinguish the entries on the basis of `sqrt(X^2 + Y^2 + Z^2)` is greater or smaller than `100*sqrt(3)/2`. Count the entries of each of the two groups 

2\.d Make `n` copies of data.txt (with `n` an input parameter of the script), where the i-th copy has all the numbers divided by i (with `1<=i<=n`).

In [ ]:
#!/bin/bash
grep -v "^#" data.csv | sed -e "s/,//g" > data.txt
grep -c "[0-9]*[02468]" data.txt

gc=0
lc=0
total_lines=$(wc -l data.txt | cut -f1 -d' ')

for ((i=1; i <= $total_lines; i++))
do
    x=$(sed -n "${i}p" data.txt | cut -d " " -f1)
    y=$(sed -n "${i}p" data.txt | cut -d " " -f2)
    z=$(sed -n "${i}p" data.txt | cut -d " " -f3)
    #echo "$x $y $z"
    square_sum=$(echo "sqrt($x^2 + $y^2 + $z^2)" | bc)
    if (( $(echo "$square_sum > 100*sqrt(3)/2" | bc -l) )) ; then
        gc=$((gc+1))
    elif (( $(echo "$square_sum < 100*sqrt(3)/2" | bc -l) )) ; then 
        lc=$((lc+1))
    fi
done
echo "gc=$gc lc=$lc"

echo "nCopies?"
read input
for ((nCopy=1; nCopy <= input; nCopy++))
do
    for (( line=1; line <= $(wc -l < data.txt); line++ ))
    do
        for word in $( sed -n "${line}p" data.txt )
        do
            num=$( echo "scale=3; $word / $nCopy" | bc)
            echo -n "$num " >> "data${nCopy}.txt"
        done
        echo >> "data${nCopy}.txt"
    done
done